# import library and api_key from openAI

In [ ]:
# Author: Yan CONG
# import libraries
import pandas as pd
import numpy as np
import math
import os
import re

# GPT3 parameters

In [ ]:
class GPT:
    """The main class for a user to interface with the OpenAI API.

    A user can add examples and set parameters of the API request.
    """
    def __init__(self,
                 engine='text-davinci-001', 
                 temperature=0.5,
                 max_tokens=100,
                 logprobs=5, 
                 input_prefix="input: ",
                 input_suffix="\n",
                 output_prefix="output: ",
                 output_suffix="\n\n",
                 append_output_prefix_to_query=False):
        self.examples = {}
        self.engine = engine
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.input_prefix = input_prefix
        self.input_suffix = input_suffix
        self.output_prefix = output_prefix
        self.output_suffix = output_suffix
        self.append_output_prefix_to_query = append_output_prefix_to_query
        self.stop = (output_suffix + input_prefix).strip()

    def add_example(self, ex):
        """Adds an example to the object.

        Example must be an instance of the Example class.
        """
        assert isinstance(ex, Example), "Please create an Example object."
        self.examples[ex.get_id()] = ex

    def delete_example(self, id):
        """Delete example with the specific id."""
        if id in self.examples:
            del self.examples[id]

    def get_example(self, id):
        """Get a single example."""
        return self.examples.get(id, None)

    def get_all_examples(self):
        """Returns all examples as a list of dicts."""
        return {k: v.as_dict() for k, v in self.examples.items()}

    def get_prime_text(self):
        """Formats all examples to prime the model."""
        return "".join(
            [self.format_example(ex) for ex in self.examples.values()])

    def get_engine(self):
        """Returns the engine specified for the API."""
        return self.engine

    def get_temperature(self):
        """Returns the temperature specified for the API."""
        return self.temperature

    def get_max_tokens(self):
        """Returns the max tokens specified for the API."""
        return self.max_tokens

    def craft_query(self, prompt):
        """Creates the query for the API request."""
        q = self.get_prime_text(
        ) + self.input_prefix + prompt + self.input_suffix
        if self.append_output_prefix_to_query:
            q = q + self.output_prefix

        return q 
    
    
    def submit_request(self, prompt):
        """Calls the OpenAI API with the specified parameters."""
        response = openai.Completion.create(engine=self.get_engine(),
                                            prompt=self.craft_query(prompt),
                                            max_tokens=self.get_max_tokens(),
                                            temperature=self.get_temperature(),
                                            logprobs=3, #yan added otherwise default null
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            stop=self.stop)
        return response

    def get_top_reply(self, prompt):
        """Obtains the best result as returned by the API."""
        response = self.submit_request(prompt)
        return response['choices'][0]['text']

    #yan added
    def get_logprobs(self, prompt):
        """Obtains the logprobs as returned by the API."""
        response = self.submit_request(prompt)
        return response['choices'][0]['logprobs']      

    def format_example(self, ex):
        """Formats the input, output pair."""
        return self.input_prefix + ex.get_input(
        ) + self.input_suffix + self.output_prefix + ex.get_output(
        ) + self.output_suffix

# Process GPT3 outputs

In [ ]:
def logprobs_to_percent(x):
    for k,v in x.items():
        p = 100*np.e**v
        x[k] = p
    return x


# Starting here: call func and class

In [ ]:
# the GPT class parameters setting will get updated here
gpt = GPT(engine="text-davinci-001", #old davinci
          temperature=0, #if we increase the temperature, it takes non-optimal answers. 
          logprobs=3, 
          max_tokens=30) #The maximum number of tokens to generate in the completion

# Run datasets

## Run scalar implicature cw quantifier

In [ ]:
for i in df.index:
    prompt = df['cw'][i] + df['post_cw'][i].strip('.') + ";"
    r = gpt.submit_request(prompt)
    pilot = pd.DataFrame(r['choices'][0]['logprobs'])

    pilot["%_token_logprobs"] = pilot["token_logprobs"].apply(lambda x: 100*np.e**x)
    pilot["%_top_logprobs"] = pilot["top_logprobs"].apply(lambda x: logprobs_to_percent(x))

    pilot['cw'] = df['cw'][i]
    pilot['post_cw'] = df['post_cw'][i]

    pilot.to_csv(str(i) + '.csv')

# Prep for data analysis

## Prep Manner implicature stats

In [ ]:
df = df[df['mean_%_token_logprobs']>0]

In [ ]:
df['condition'] = ''
df['prediction'] = ''
for i in df.index:
    if i == 0 or i % 2 == 0:
        df['condition'][i] = 'experimental'
        df['prediction'][i] = 'good'
    else:
        df['condition'][i] = 'baseline'
        df['prediction'][i] = 'bad'

In [ ]:
df['accuracy'] = ''
for i in df.index:
    if i%2 == 0:
        try:
            if df['mean_%_token_logprobs'][i] >= df['mean_%_token_logprobs'][i+1]:
                df['accuracy'][i] = '1'
            else:
                df['accuracy'][i] = '0'
        except:
            pass
    else:
        df['accuracy'][i] = 'na'